# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
findspark.init('C:\\spark')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Movies  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"msd/kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 


## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [3]:
from operator import add

In [4]:
song_rdd = sc.textFile(r"kaggle_songs.txt").map(lambda line: line.split(" "))
song_map = song_rdd.map(lambda x:(x[0],x[1]))
count = song_rdd.map(lambda x:1).reduce(add)
count
#print(song_map.take(3))

386213

In [9]:
triplet_map = triplet_rdd.map(lambda x:(x[1],(x[0],x[2])))
join0 = song_map.leftOuterJoin(triplet_map)
print(join0.take(2))
final = join0.map(lambda x:(x[1]))

print(final.take(3))
def liked(col):
    if(col[1] != None):
        return True
    return False
replace = final.filter(liked).map(lambda x:(x[1][0],x[0],x[1][1]))

replace.collect()

[('SOAAALJ12AB01828B4', ('12', None)), ('SOAAAQN12AB01856D3', ('16', ('32cf63cf65787ce7e72fc7fda6ee585979af6582', '1')))]
[('12', None), ('16', ('32cf63cf65787ce7e72fc7fda6ee585979af6582', '1')), ('16', ('d1b88940eabd4fab860edf68f0f0842e4f902c78', '1'))]


[('32cf63cf65787ce7e72fc7fda6ee585979af6582', '16', '1'),
 ('d1b88940eabd4fab860edf68f0f0842e4f902c78', '16', '1'),
 ('140be58d1ea6d645ed7b317f61197b97975ac1ba', '29', '1'),
 ('bb2e92130a312d9f9e5754f05b36ca85505656b0', '42', '1'),
 ('12e71abacd59cd98b403ffc0444b19dc3ba0b0d6', '48', '1'),
 ('7724c3fd81ac74aa4705a66e57be24bd1401be38', '48', '1'),
 ('8b14cb7563e7355c645efd6cea95077f6c1061f3', '48', '1'),
 ('4ed80c2caf9707dfd0cd1135d6d940910929ebac', '48', '2'),
 ('a6c9408097565d01ce8c39d38fc1c75a35959821', '48', '3'),
 ('6ac4168642e44e3d1e82ae813b1ddefe7833ef31', '52', '2'),
 ('350aae3e2903956a2c62af669ec8e5a4458af2f7', '52', '1'),
 ('e93b74d7e7e8cc3ec0880d8f2c2616f82cb38f91', '52', '3'),
 ('24f629e50f599a41eadbb014f70817ae95490b58', '54', '1'),
 ('91023f08c63f4ed50061338590d94fc277f6e065', '54', '5'),
 ('9d693d6a8a2e09f3de41d0e7eedd4b590f96b99b', '54', '1'),
 ('2cb9af8ee2be814260dcb00f8b09e384ece09c86', '54', '1'),
 ('480b5be680d04268a5af294c3320c69b6a92dbee', '54', '5'),
 ('810d0a292d0

In [8]:
nore = replace.map(lambda x:(x[1],x))
noredun = nore.reduceByKey(lambda x,y:x)
count1=noredun.map(lambda x:1).reduce(add)
outcome = count-count1
print("The number of songs without user history is ", outcome )

The number of songs without user history is  223007


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [37]:
user_rdd = sc.textFile(r"kaggle_users.txt") \
    .map(lambda line: line.split("\n")) 
usermap = user_rdd.map(lambda x: x[0])
# usermap.take(5)
usertimes = triplet_rdd.map(lambda x:(x[0],int(x[2]))).reduceByKey(add)
print(usertimes.take(5))
usertimes_map =usertimes.map(lambda x:(x[0],x[1]))
# times = triplet_rdd.map(lambda x:(x[1],1)).reduceByKey(add)
# print(times.take(5))
# time = triplet_rdd.map(lambda x:(x[1],int(x[2]))).reduceByKey(add)
# print(time.take(5))

[('d7083f5e1d50c264277d624340edaaf3dc16095b', 17), ('d68dc6fc25248234590d7668a11e3335534ae4b4', 6), ('fdf6afb5daefb42774617cf223475c6013969724', 10), ('10cbcd627472477dfbec90fb75017f8df6ce84ec', 9), ('30e4a688e6fc9c8bfe55998af3996a909ae34449', 8)]


In [38]:
replace_map = replace.map(lambda x:(x[0],(x[1],x[2])))
join1 = usertimes_map.leftOuterJoin(replace_map)
# print(join1.take(3))
ratings = join1.map(lambda x:(x[0],x[1][1][0],int(x[1][1][1])/x[1][0]))
print(ratings.take(5))
# join1 = times.leftOuterJoin(time)
# # print(join1.take(3))
# ratings = join1.map(lambda x:(x[0],x[1][1]/x[1][0]))
# # print(ratings.take(3))
# ratings.collect()
ratings.collect()

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 0.11363636363636363), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 0.18181818181818182), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 0.06818181818181818), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 0.022727272727272728), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 0.1590909090909091)]


[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 0.11363636363636363),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 0.18181818181818182),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 0.06818181818181818),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 0.022727272727272728),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 0.1590909090909091),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '253868', 0.022727272727272728),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '280679', 0.045454545454545456),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '294531', 0.022727272727272728),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '357202', 0.022727272727272728),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '346138', 0.045454545454545456),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '165401', 0.045454545454545456),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '176139', 0.022727272727272728),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '275127', 0.0

In [39]:
import math
final_rating = ratings.map(lambda x:(x[0],x[1],10+math.log(x[2])))
final_rating.collect()

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 7.825248278515839),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 8.295251907761575),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 7.3144226547498485),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 6.2158103660817385),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 8.161720515137052),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '253868', 6.2158103660817385),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '280679', 6.908957546641684),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '294531', 6.2158103660817385),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '357202', 6.2158103660817385),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '346138', 6.908957546641684),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '165401', 6.908957546641684),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '176139', 6.2158103660817385),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '275127', 6.908957546641684),
 ('6530c

## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [22]:
triplet_map1 = triplet_rdd.map(lambda x:(x[1],(x[0],x[2])))
ratings_map = ratings.map(lambda x:(x[0],x[1]))
join2 = triplet_map1.leftOuterJoin(ratings_map)
join2.take(5)
def liked(col):
    if(col[1] != None and col[0] != None):
        return True
    return False
whole = join2.filter(liked).map(lambda x:(x[1][0][0],x[0],x[1][1],x[1][0][1]))

whole.take(5)

[('fd50c4007b68a3737fe052d5a4f78ce8aa117f3d',
  'SOFLJQZ12A6D4FADA6',
  2.959145427286357,
  '1'),
 ('248378ac27e1745d6a9d59392b7dc5b02a6186a6',
  'SOFLJQZ12A6D4FADA6',
  2.959145427286357,
  '2'),
 ('dc03114dc0a52c407d1548a403c8af85a82fcb4a',
  'SOFLJQZ12A6D4FADA6',
  2.959145427286357,
  '1'),
 ('91ebba4aec748ba839c9f64ace1a2f22228a16dd',
  'SOFLJQZ12A6D4FADA6',
  2.959145427286357,
  '1'),
 ('e326c4b9fe3659ec1dc3af53fd7e0893809dafbc',
  'SOFLJQZ12A6D4FADA6',
  2.959145427286357,
  '2')]

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [ ]:
#1.Compute cosine similarity between all pairs of users.
